In [1]:
import numpy as np
import pandas as pd
import os
import gc
import warnings
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# Holidays and Events

In [2]:
holidays = pd.read_csv("../input/store-sales-time-series-forecasting/holidays_events.csv")
holidays["date"] = pd.to_datetime(holidays.date)

In [3]:
holidays

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False
...,...,...,...,...,...,...
345,2017-12-22,Additional,National,Ecuador,Navidad-3,False
346,2017-12-23,Additional,National,Ecuador,Navidad-2,False
347,2017-12-24,Additional,National,Ecuador,Navidad-1,False
348,2017-12-25,Holiday,National,Ecuador,Navidad,False


## Transferred Holidays

In [4]:
tr1 = holidays[(holidays.type == "Holiday") & (holidays.transferred == True)].drop("transferred", axis = 1).reset_index(drop = True)
tr2 = holidays[(holidays.type == "Transfer")].drop("transferred", axis = 1).reset_index(drop = True)
tr = pd.concat([tr1,tr2], axis = 1)
tr = tr.iloc[:, [5,1,2,3,4]]

In [5]:
tr

,date,type,locale,locale_name,description
0,2012-10-12,Holiday,National,Ecuador,Independencia de Guayaquil
1,2013-10-11,Holiday,National,Ecuador,Independencia de Guayaquil
2,2014-10-10,Holiday,National,Ecuador,Independencia de Guayaquil
3,2016-05-27,Holiday,National,Ecuador,Batalla de Pichincha
4,2016-07-24,Holiday,Local,Guayaquil,Fundacion de Guayaquil
5,2016-08-12,Holiday,National,Ecuador,Primer Grito de Independencia
6,2017-01-02,Holiday,National,Ecuador,Primer dia del ano
7,2017-04-13,Holiday,Local,Cuenca,Fundacion de Cuenca
8,2017-05-26,Holiday,National,Ecuador,Batalla de Pichincha
9,2017-08-11,Holiday,National,Ecuador,Primer Grito de Independencia


In [6]:
holidays = holidays[(holidays.transferred == False) & (holidays.type != "Transfer")].drop("transferred", axis = 1)
holidays = holidays.append(tr).reset_index(drop = True)

In [7]:
holidays

,date,type,locale,locale_name,description
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba
...,...,...,...,...,...
333,2017-04-13,Holiday,Local,Cuenca,Fundacion de Cuenca
334,2017-05-26,Holiday,National,Ecuador,Batalla de Pichincha
335,2017-08-11,Holiday,National,Ecuador,Primer Grito de Independencia
336,2017-09-29,Holiday,Local,Ibarra,Fundacion de Ibarra


## Additional Holidays

In [8]:
holidays["description"] = holidays["description"].str.replace("-", "").str.replace("+", "").str.replace('\d+', '')
holidays["type"] = np.where(holidays["type"] == "Additional", "Holiday", holidays["type"])
holidays

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


,date,type,locale,locale_name,description
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba
...,...,...,...,...,...
333,2017-04-13,Holiday,Local,Cuenca,Fundacion de Cuenca
334,2017-05-26,Holiday,National,Ecuador,Batalla de Pichincha
335,2017-08-11,Holiday,National,Ecuador,Primer Grito de Independencia
336,2017-09-29,Holiday,Local,Ibarra,Fundacion de Ibarra


## Bridge Holidays

In [9]:
holidays["description"] = holidays["description"].str.replace("puente ", "")
holidays["type"] = np.where(holidays["type"] == "Bridge", "Holidays", holidays["type"])
holidays

,date,type,locale,locale_name,description
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba
...,...,...,...,...,...
333,2017-04-13,Holiday,Local,Cuenca,Fundacion de Cuenca
334,2017-05-26,Holiday,National,Ecuador,Batalla de Pichincha
335,2017-08-11,Holiday,National,Ecuador,Primer Grito de Independencia
336,2017-09-29,Holiday,Local,Ibarra,Fundacion de Ibarra


## Work Day Holidays, that is meant to payback the Bridge

In [10]:
work_day = holidays[holidays.type == "Work Day"]
holidays = holidays[holidays.type != "Work Day"]

## Events are national

In [11]:
events = holidays[holidays.type == "Event"].drop(["type", "locale", "locale_name"], axis=1).rename({"description": "events"}, axis=1)
events.head()

,date,events
53,2013-05-12,Dia de la Madre
99,2014-05-11,Dia de la Madre
102,2014-06-12,Inauguracion Mundial de futbol Brasil
103,2014-06-15,Mundial de futbol Brasil: EcuadorSuiza
104,2014-06-20,Mundial de futbol Brasil: EcuadorHonduras


In [12]:
holidays = holidays[holidays.type != "Event"].drop("type", axis=1)
regional = holidays[holidays.locale == "Regional"].rename({"locale_name": "state", "description": "holiday_regional"},axis=1).drop("locale", axis=1).drop_duplicates()
regional

,date,state,holiday_regional
1,2012-04-01,Cotopaxi,Provincializacion de Cotopaxi
7,2012-06-25,Imbabura,Provincializacion de Imbabura
21,2012-11-06,Santo Domingo de los Tsachilas,Provincializacion de Santo Domingo
22,2012-11-07,Santa Elena,Provincializacion Santa Elena
45,2013-04-01,Cotopaxi,Provincializacion de Cotopaxi
56,2013-06-25,Imbabura,Provincializacion de Imbabura
72,2013-11-06,Santo Domingo de los Tsachilas,Provincializacion de Santo Domingo
73,2013-11-07,Santa Elena,Provincializacion Santa Elena
92,2014-04-01,Cotopaxi,Provincializacion de Cotopaxi
108,2014-06-25,Imbabura,Provincializacion de Imbabura


In [13]:
national = holidays[holidays.locale == "National"].rename({"description": "holiday_national"}, axis=1).drop(["locale", "locale_name"], axis=1).drop_duplicates()
national

,date,holiday_national
14,2012-08-10,Primer Grito de Independencia
19,2012-11-02,Dia de Difuntos
20,2012-11-03,Independencia de Cuenca
29,2012-12-21,Navidad
31,2012-12-22,Navidad
...,...,...
329,2016-05-27,Batalla de Pichincha
331,2016-08-12,Primer Grito de Independencia
332,2017-01-02,Primer dia del ano
334,2017-05-26,Batalla de Pichincha


In [14]:
local = holidays[holidays.locale == "Local"].rename({"description":"holiday_local", "locale_name":"city"}, axis = 1).drop("locale", axis = 1).drop_duplicates()
local

,date,city,holiday_local
0,2012-03-02,Manta,Fundacion de Manta
2,2012-04-12,Cuenca,Fundacion de Cuenca
3,2012-04-14,Libertad,Cantonizacion de Libertad
4,2012-04-21,Riobamba,Cantonizacion de Riobamba
5,2012-05-12,Puyo,Cantonizacion del Puyo
...,...,...,...
318,2017-12-08,Loja,Fundacion de Loja
320,2017-12-22,Salinas,Cantonizacion de Salinas
333,2017-04-13,Cuenca,Fundacion de Cuenca
336,2017-09-29,Ibarra,Fundacion de Ibarra


## Merge train, test, stores data

In [15]:
train = pd.read_csv('../input/store-sales-time-series-forecasting/train.csv')
test = pd.read_csv('../input/store-sales-time-series-forecasting/test.csv')
stores = pd.read_csv('../input/store-sales-time-series-forecasting/stores.csv')

In [16]:
train["date"] = pd.to_datetime(train.date)
test["date"] = pd.to_datetime(test.date)

# Data types
train.onpromotion = train.onpromotion.astype("float16")
train.sales = train.sales.astype("float32")
stores.cluster = stores.cluster.astype("int8")

In [17]:
d = pd.merge(train.append(test), stores)
d["store_nbr"] = d["store_nbr"].astype("int8")

In [18]:
d

,id,date,store_nbr,family,sales,onpromotion,city,state,type,cluster
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0.0,Quito,Pichincha,D,13
1,1,2013-01-01,1,BABY CARE,0.0,0.0,Quito,Pichincha,D,13
2,2,2013-01-01,1,BEAUTY,0.0,0.0,Quito,Pichincha,D,13
3,3,2013-01-01,1,BEVERAGES,0.0,0.0,Quito,Pichincha,D,13
4,4,2013-01-01,1,BOOKS,0.0,0.0,Quito,Pichincha,D,13
...,...,...,...,...,...,...,...,...,...,...
3029395,3029395,2017-08-31,9,POULTRY,NaN,1.0,Quito,Pichincha,B,6
3029396,3029396,2017-08-31,9,PREPARED FOODS,NaN,0.0,Quito,Pichincha,B,6
3029397,3029397,2017-08-31,9,PRODUCE,NaN,1.0,Quito,Pichincha,B,6
3029398,3029398,2017-08-31,9,SCHOOL AND OFFICE SUPPLIES,NaN,9.0,Quito,Pichincha,B,6


In [19]:
d = pd.merge(d, national, how = "left")
d = pd.merge(d, regional, how = "left", on = ["date", "state"])
d = pd.merge(d, local, how = "left", on = ["date", "city"])

d = pd.merge(d,  work_day[["date", "type"]].rename({"type":"IsWorkDay"}, axis = 1),how = "left")

In [20]:
d

,id,date,store_nbr,family,sales,onpromotion,city,state,type,cluster,holiday_national,holiday_regional,holiday_local,IsWorkDay
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0.0,Quito,Pichincha,D,13,Primer dia del ano,NaN,NaN,NaN
1,1,2013-01-01,1,BABY CARE,0.0,0.0,Quito,Pichincha,D,13,Primer dia del ano,NaN,NaN,NaN
2,2,2013-01-01,1,BEAUTY,0.0,0.0,Quito,Pichincha,D,13,Primer dia del ano,NaN,NaN,NaN
3,3,2013-01-01,1,BEVERAGES,0.0,0.0,Quito,Pichincha,D,13,Primer dia del ano,NaN,NaN,NaN
4,4,2013-01-01,1,BOOKS,0.0,0.0,Quito,Pichincha,D,13,Primer dia del ano,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3031177,3029395,2017-08-31,9,POULTRY,NaN,1.0,Quito,Pichincha,B,6,NaN,NaN,NaN,NaN
3031178,3029396,2017-08-31,9,PREPARED FOODS,NaN,0.0,Quito,Pichincha,B,6,NaN,NaN,NaN,NaN
3031179,3029397,2017-08-31,9,PRODUCE,NaN,1.0,Quito,Pichincha,B,6,NaN,NaN,NaN,NaN
3031180,3029398,2017-08-31,9,SCHOOL AND OFFICE SUPPLIES,NaN,9.0,Quito,Pichincha,B,6,NaN,NaN,NaN,NaN


## EVENTS

In [21]:
events["events"] = np.where(events.events.str.contains("futbol"), "Futbol", events.events)
events

,date,events
53,2013-05-12,Dia de la Madre
99,2014-05-11,Dia de la Madre
102,2014-06-12,Futbol
103,2014-06-15,Futbol
104,2014-06-20,Futbol
109,2014-06-25,Futbol
110,2014-06-28,Futbol
111,2014-06-29,Futbol
112,2014-06-30,Futbol
113,2014-07-01,Futbol


## One Hot Encoder

In [22]:
def one_hot_encoder(df, nan_as_Category=True):
    original_columns = list(df.columns)
    categorical_columns = df.select_dtypes(["category", "object"]).columns.tolist()
    df = pd.get_dummies(df, columns=categorical_columns, dummy_na=nan_as_Category)
    new_columns = [c for c in df.columns if c not in original_columns]
    df.columns = df.columns.str.replace(" ", "_")
    return df, df.columns.tolist()

In [23]:
events, events_cat = one_hot_encoder(events, nan_as_Category=False)
events["events_Dia_de_la_Madre"] = np.where(events.date == "2016-05-08", 1, events["events_Dia_de_la_Madre"])
events = events.drop(239)
events

,date,events_Black_Friday,events_Cyber_Monday,events_Dia_de_la_Madre,events_Futbol,events_Terremoto_Manabi
53,2013-05-12,0,0,1,0,0
99,2014-05-11,0,0,1,0,0
102,2014-06-12,0,0,0,1,0
103,2014-06-15,0,0,0,1,0
104,2014-06-20,0,0,0,1,0
109,2014-06-25,0,0,0,1,0
110,2014-06-28,0,0,0,1,0
111,2014-06-29,0,0,0,1,0
112,2014-06-30,0,0,0,1,0
113,2014-07-01,0,0,0,1,0


In [24]:
d = pd.merge(d, events, how="left")
d[events_cat] = d[events_cat].fillna(0)
d

,id,date,store_nbr,family,sales,onpromotion,city,state,type,cluster,holiday_national,holiday_regional,holiday_local,IsWorkDay,events_Black_Friday,events_Cyber_Monday,events_Dia_de_la_Madre,events_Futbol,events_Terremoto_Manabi
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0.0,Quito,Pichincha,D,13,Primer dia del ano,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0
1,1,2013-01-01,1,BABY CARE,0.0,0.0,Quito,Pichincha,D,13,Primer dia del ano,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0
2,2,2013-01-01,1,BEAUTY,0.0,0.0,Quito,Pichincha,D,13,Primer dia del ano,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0
3,3,2013-01-01,1,BEVERAGES,0.0,0.0,Quito,Pichincha,D,13,Primer dia del ano,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0
4,4,2013-01-01,1,BOOKS,0.0,0.0,Quito,Pichincha,D,13,Primer dia del ano,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3031177,3029395,2017-08-31,9,POULTRY,NaN,1.0,Quito,Pichincha,B,6,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0
3031178,3029396,2017-08-31,9,PREPARED FOODS,NaN,0.0,Quito,Pichincha,B,6,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0
3031179,3029397,2017-08-31,9,PRODUCE,NaN,1.0,Quito,Pichincha,B,6,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0
3031180,3029398,2017-08-31,9,SCHOOL AND OFFICE SUPPLIES,NaN,9.0,Quito,Pichincha,B,6,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0


## New features

In [25]:
d["holiday_national_binary"] = np.where(d.holiday_national.notnull(), 1, 0)
d["holiday_local_binary"] = np.where(d.holiday_local.notnull(), 1, 0)
d["holiday_regional_binary"] = np.where(d.holiday_regional.notnull(), 1, 0)

d["national_independence"] = np.where(d.holiday_national.isin(['Batalla de Pichincha',  
                                                               'Independencia de Cuenca', 
                                                               'Independencia de Guayaquil', 
                                                               'Independencia de Guayaquil', 
                                                               'Primer Grito de Independencia']), 1, 0)
d["local_cantonizacio"] = np.where(d.holiday_local.str.contains("Cantonizacio"), 1, 0)
d["local_fundacion"] = np.where(d.holiday_local.str.contains("Fundacion"), 1, 0)
d["local_independencia"] = np.where(d.holiday_local.str.contains("Independencia"), 1, 0)


In [26]:
holidays, holidays_cat = one_hot_encoder(d[["holiday_national","holiday_regional","holiday_local"]], nan_as_Category=False)
d = pd.concat([d.drop(["holiday_national","holiday_regional","holiday_local"], axis = 1),holidays], axis = 1)
d

,id,date,store_nbr,family,sales,onpromotion,city,state,type,cluster,...,holiday_local_Fundacion_de_Ibarra,holiday_local_Fundacion_de_Loja,holiday_local_Fundacion_de_Machala,holiday_local_Fundacion_de_Manta,holiday_local_Fundacion_de_Quito,holiday_local_Fundacion_de_Riobamba,holiday_local_Fundacion_de_Santo_Domingo,holiday_local_Independencia_de_Ambato,holiday_local_Independencia_de_Guaranda,holiday_local_Independencia_de_Latacunga
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0.0,Quito,Pichincha,D,13,...,0,0,0,0,0,0,0,0,0,0
1,1,2013-01-01,1,BABY CARE,0.0,0.0,Quito,Pichincha,D,13,...,0,0,0,0,0,0,0,0,0,0
2,2,2013-01-01,1,BEAUTY,0.0,0.0,Quito,Pichincha,D,13,...,0,0,0,0,0,0,0,0,0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0.0,Quito,Pichincha,D,13,...,0,0,0,0,0,0,0,0,0,0
4,4,2013-01-01,1,BOOKS,0.0,0.0,Quito,Pichincha,D,13,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3031177,3029395,2017-08-31,9,POULTRY,NaN,1.0,Quito,Pichincha,B,6,...,0,0,0,0,0,0,0,0,0,0
3031178,3029396,2017-08-31,9,PREPARED FOODS,NaN,0.0,Quito,Pichincha,B,6,...,0,0,0,0,0,0,0,0,0,0
3031179,3029397,2017-08-31,9,PRODUCE,NaN,1.0,Quito,Pichincha,B,6,...,0,0,0,0,0,0,0,0,0,0
3031180,3029398,2017-08-31,9,SCHOOL AND OFFICE SUPPLIES,NaN,9.0,Quito,Pichincha,B,6,...,0,0,0,0,0,0,0,0,0,0


In [27]:
he_cols = d.columns[d.columns.str.startswith("events")].tolist() + d.columns[d.columns.str.startswith("holiday")].tolist() + d.columns[d.columns.str.startswith("national")].tolist()+ d.columns[d.columns.str.startswith("local")].tolist()
d[he_cols] = d[he_cols].astype("int8")

In [28]:
d[["family", "city", "state", "type"]] = d[["family", "city", "state", "type"]].astype("category")

del holidays, holidays_cat, work_day, local, regional, national, events, events_cat, tr, tr1, tr2, he_cols
gc.collect()

d.head(10)

,id,date,store_nbr,family,sales,onpromotion,city,state,type,cluster,...,holiday_local_Fundacion_de_Ibarra,holiday_local_Fundacion_de_Loja,holiday_local_Fundacion_de_Machala,holiday_local_Fundacion_de_Manta,holiday_local_Fundacion_de_Quito,holiday_local_Fundacion_de_Riobamba,holiday_local_Fundacion_de_Santo_Domingo,holiday_local_Independencia_de_Ambato,holiday_local_Independencia_de_Guaranda,holiday_local_Independencia_de_Latacunga
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0.0,Quito,Pichincha,D,13,...,0,0,0,0,0,0,0,0,0,0
1,1,2013-01-01,1,BABY CARE,0.0,0.0,Quito,Pichincha,D,13,...,0,0,0,0,0,0,0,0,0,0
2,2,2013-01-01,1,BEAUTY,0.0,0.0,Quito,Pichincha,D,13,...,0,0,0,0,0,0,0,0,0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0.0,Quito,Pichincha,D,13,...,0,0,0,0,0,0,0,0,0,0
4,4,2013-01-01,1,BOOKS,0.0,0.0,Quito,Pichincha,D,13,...,0,0,0,0,0,0,0,0,0,0
5,5,2013-01-01,1,BREAD/BAKERY,0.0,0.0,Quito,Pichincha,D,13,...,0,0,0,0,0,0,0,0,0,0
6,6,2013-01-01,1,CELEBRATION,0.0,0.0,Quito,Pichincha,D,13,...,0,0,0,0,0,0,0,0,0,0
7,7,2013-01-01,1,CLEANING,0.0,0.0,Quito,Pichincha,D,13,...,0,0,0,0,0,0,0,0,0,0
8,8,2013-01-01,1,DAIRY,0.0,0.0,Quito,Pichincha,D,13,...,0,0,0,0,0,0,0,0,0,0
9,9,2013-01-01,1,DELI,0.0,0.0,Quito,Pichincha,D,13,...,0,0,0,0,0,0,0,0,0,0


# AB test
**Let's apply an AB test to Events and Holidays features. Are they statistically significant? Also it can be a good way for first feature selection.**

- *H0: The sales are equal* **(M1 = M2)**
- *H1: The sales are not equal* **(M1 != M2)**

In [29]:
def AB_Test(dataframe, group, target):
    
    from scipy.stats import shapiro
    import scipy.stats as stats
    
    groupA = dataframe[dataframe[group] == 1][target]
    groupB = dataframe[dataframe[group] == 0][target]
    
    ntA = shapiro(groupA)[1] < 0.05
    ntB = shapiro(groupB)[1] < 0.05

    if (ntA == False) & (ntB == False): 
        leveneTest = stats.levene(groupA, groupB)[1] < 0.05
        if leveneTest == False:
            ttest = stats.ttest_ind(groupA, groupB, equal_var=True)[1]
        else:
            ttest = stats.ttest_ind(groupA, groupB, equal_var=False)[1]

    else:
        ttest = stats.mannwhitneyu(groupA, groupB)[1] 

    temp = pd.DataFrame({"AB Hypothesis":[ttest < 0.05],"p-value":[ttest]})
    temp["Test Type"] = np.where((ntA == False) & (ntB == False), "Parametric", "Non-Parametric")
    temp["AB Hypothesis"] = np.where(temp["AB Hypothesis"] == False, "Fail to Reject H0", "Reject H0")
    temp["Comment"] = np.where(temp["AB Hypothesis"] == "Fail to Reject H0", "A/B groups are similar!", "A/B groups are not similar!")
    temp["Feature"] = group
    
    temp["GroupA_mean"] = groupA.mean()
    temp["GroupB_mean"] = groupB.mean()
    temp["GroupA_median"] = groupA.median()
    temp["GroupB_median"] = groupB.median()
    
    if (ntA == False) & (ntB == False):
        temp["Homogeneity"] = np.where(leveneTest == False, "Yes", "No")
        temp = temp[["Feature",
                     "Test Type", 
                     "Homogeneity",
                     "AB Hypothesis",
                     "p-value", 
                     "Comment", 
                     "GroupA_mean", 
                     "GroupB_mean", 
                     "GroupA_median", 
                     "GroupB_median"]]
    else:
        temp = temp[["Feature",
                     "Test Type",
                     "AB Hypothesis", 
                     "p-value", "Comment", 
                     "GroupA_mean", 
                     "GroupB_mean", 
                     "GroupA_median", "GroupB_median"]]
    
    return temp

## Apply A/B Testing

In [30]:
he_cols = d.columns[d.columns.str.startswith("events")].tolist() + d.columns[d.columns.str.startswith("holiday")].tolist() + d.columns[d.columns.str.startswith("national")].tolist()+ d.columns[d.columns.str.startswith("local")].tolist()
ab = []
for i in he_cols:
    ab.append(AB_Test(dataframe=d[d.sales.notnull()], group = i, target = "sales"))
ab = pd.concat(ab)
ab

/opt/conda/lib/python3.7/site-packages/scipy/stats/morestats.py:1760: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")
/opt/conda/lib/python3.7/site-packages/scipy/stats/morestats.py:1760: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")
/opt/conda/lib/python3.7/site-packages/scipy/stats/morestats.py:1760: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")
/opt/conda/lib/python3.7/site-packages/scipy/stats/morestats.py:1760: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")
/opt/conda/lib/python3.7/site-packages/scipy/stats/morestats.py:1760: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")
/opt/conda/lib/python3.7/site-packages/scipy/stats/morestats.py:1760: 

,Feature,Test Type,AB Hypothesis,p-value,Comment,GroupA_mean,GroupB_mean,GroupA_median,GroupB_median
0,events_Black_Friday,Non-Parametric,Reject H0,2.815219e-20,A/B groups are not similar!,363.360718,357.843964,17.000000,11.0
0,events_Cyber_Monday,Non-Parametric,Reject H0,3.748090e-27,A/B groups are not similar!,436.220245,357.713989,17.000000,11.0
0,events_Dia_de_la_Madre,Non-Parametric,Reject H0,2.407437e-04,A/B groups are not similar!,348.536499,357.881622,9.000000,11.0
0,events_Futbol,Non-Parametric,Reject H0,4.075879e-94,A/B groups are not similar!,310.965149,358.246674,5.000000,11.0
0,events_Terremoto_Manabi,Non-Parametric,Reject H0,0.000000e+00,A/B groups are not similar!,494.904083,355.285187,23.000000,11.0
0,holiday_national_binary,Non-Parametric,Reject H0,4.116942e-03,A/B groups are not similar!,425.269287,354.537537,11.000000,11.0
0,holiday_local_binary,Non-Parametric,Reject H0,7.264108e-11,A/B groups are not similar!,427.115082,357.578735,14.000000,11.0
0,holiday_regional_binary,Non-Parametric,Reject H0,3.558854e-04,A/B groups are not similar!,229.725433,357.897675,6.000000,11.0
0,holiday_national_Batalla_de_Pichincha,Non-Parametric,Fail to Reject H0,6.504450e-01,A/B groups are similar!,391.726776,357.752991,12.000000,11.0
0,holiday_national_Carnaval,Non-Parametric,Reject H0,1.408976e-02,A/B groups are not similar!,332.882568,358.002930,10.000000,11.0


# Time Related Features

In [31]:
d['month'] = d.date.dt.month.astype('int8')
d['day_of_month'] = d.date.dt.day.astype('int8')
d['day_of_year'] = d.date.dt.dayofyear.astype('int16')
d['day_of_week'] = (d.date.dt.dayofweek + 1).astype("int8")

d['week_of_month'] = (d.date.apply(lambda d: (d.day-1) // 7 + 1)).astype('int8')
d['week_of_year'] = (d.date.dt.weekofyear).astype('int8')

d['year'] = d.date.dt.year.astype('int32')
d['is_wknd'] = (d.date.dt.weekday // 4).astype('int8')
d['quarter'] = d.date.dt.quarter.astype('int8')

d['is_month_start'] = d.date.dt.is_month_start.astype('int8')
d['is_month_end'] = d.date.dt.is_month_end.astype("int8")
d['is_quarter_start'] = d.date.dt.is_quarter_start.astype('int8')
d['is_year_start'] = d.date.dt.is_year_start.astype("int8")
d['is_year_end'] = d.date.dt.is_year_end.astype("int8")
#d

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  import sys


In [32]:
d['season'] = np.where(d.month.isin([12,1,2]), 0, 1)
d["season"] = np.where(d.month.isin([6,7,8]), 2, d["season"])
d["season"] = pd.Series(np.where(d.month.isin([9, 10, 11]), 3, d["season"])).astype("int8")

In [33]:
d["workday"] = np.where((d.holiday_national_binary == 1) | (d.holiday_local_binary==1) | (d.holiday_regional_binary==1) | (d['day_of_week'].isin([6,7])), 0, 1)
d["workday"] = pd.Series(np.where(d.IsWorkDay.notnull(), 1, d["workday"])).astype("int8")
#d.drop("IsWorkDay", axis = 1, inplace = True)

d["wageday"] = pd.Series(np.where((d['is_month_end'] == 1) | (d["day_of_month"] == 15), 1, 0)).astype("int8")
d

,id,date,store_nbr,family,sales,onpromotion,city,state,type,cluster,...,is_wknd,quarter,is_month_start,is_month_end,is_quarter_start,is_year_start,is_year_end,season,workday,wageday
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0.0,Quito,Pichincha,D,13,...,0,1,1,0,1,1,0,0,0,0
1,1,2013-01-01,1,BABY CARE,0.0,0.0,Quito,Pichincha,D,13,...,0,1,1,0,1,1,0,0,0,0
2,2,2013-01-01,1,BEAUTY,0.0,0.0,Quito,Pichincha,D,13,...,0,1,1,0,1,1,0,0,0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0.0,Quito,Pichincha,D,13,...,0,1,1,0,1,1,0,0,0,0
4,4,2013-01-01,1,BOOKS,0.0,0.0,Quito,Pichincha,D,13,...,0,1,1,0,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3031177,3029395,2017-08-31,9,POULTRY,NaN,1.0,Quito,Pichincha,B,6,...,0,3,0,1,0,0,0,2,1,1
3031178,3029396,2017-08-31,9,PREPARED FOODS,NaN,0.0,Quito,Pichincha,B,6,...,0,3,0,1,0,0,0,2,1,1
3031179,3029397,2017-08-31,9,PRODUCE,NaN,1.0,Quito,Pichincha,B,6,...,0,3,0,1,0,0,0,2,1,1
3031180,3029398,2017-08-31,9,SCHOOL AND OFFICE SUPPLIES,NaN,9.0,Quito,Pichincha,B,6,...,0,3,0,1,0,0,0,2,1,1


In [34]:
d[(d.month.isin([4,5]))].groupby(["year"]).sales.mean()

year
2013    207.826614
2014    241.403442
2015    300.688446
2016    466.803345
2017    485.797455
Name: sales, dtype: float32

## March

In [35]:
pd.pivot_table(d[(d.month.isin([3]))], index='year', columns='family', values='sales', aggfunc='mean')

family,AUTOMOTIVE,BABY CARE,BEAUTY,BEVERAGES,BOOKS,BREAD/BAKERY,CELEBRATION,CLEANING,DAIRY,DELI,...,MAGAZINES,MEATS,PERSONAL CARE,PET SUPPLIES,PLAYERS AND ELECTRONICS,POULTRY,PREPARED FOODS,PRODUCE,SCHOOL AND OFFICE SUPPLIES,SEAFOOD
year,,,,,,,,,,,,,,,,,,,,,
2013,4.560932,0.000000,2.330347,1088.542358,0.000000,354.853516,0.000000,899.277161,365.894867,195.494293,...,0.000000,313.938690,190.713257,0.000000,0.000000,190.441727,90.362183,2.411589,0.000000,23.941057
2014,6.042413,0.043011,3.086021,2646.471924,0.000000,388.987701,12.830346,1057.511353,709.575256,227.415344,...,1.170251,332.993347,231.297485,4.785543,9.594385,390.943054,97.848579,2002.596313,1.335125,23.890751
2015,6.902031,0.000000,2.538232,1385.922363,0.000000,500.669403,0.000000,1115.107544,737.664856,301.454895,...,0.000000,326.473816,289.569885,0.000000,0.000000,384.931152,95.655190,4.124851,0.000000,26.169935
2016,7.203106,0.264038,4.261051,2967.511963,0.000000,492.651520,13.970132,1057.762817,843.405029,272.667236,...,5.959976,324.644165,290.982666,5.780765,8.557945,368.022827,107.821007,2262.134521,1.922342,23.449707
2017,7.642772,0.264038,4.212067,3664.010254,0.281959,573.421265,13.947432,1197.510742,972.788513,320.441528,...,6.161888,367.350983,325.296906,8.627837,11.180407,388.023895,111.916695,2396.963135,4.925926,25.063908


## April-May

In [36]:
pd.pivot_table(d[(d.month.isin([4,5]))], index='year', columns='family', values='sales', aggfunc='mean')

family,AUTOMOTIVE,BABY CARE,BEAUTY,BEVERAGES,BOOKS,BREAD/BAKERY,CELEBRATION,CLEANING,DAIRY,DELI,...,MAGAZINES,MEATS,PERSONAL CARE,PET SUPPLIES,PLAYERS AND ELECTRONICS,POULTRY,PREPARED FOODS,PRODUCE,SCHOOL AND OFFICE SUPPLIES,SEAFOOD
year,,,,,,,,,,,,,,,,,,,,,
2013,5.014875,0.000000,2.242259,1082.952026,0.000000,359.644470,0.000000,882.671814,350.712494,201.413803,...,0.000000,324.426300,192.935638,0.000000,0.000000,190.306381,83.029396,4.597450,0.000000,21.784353
2014,5.009411,0.000000,2.277171,1273.498169,0.000000,362.751038,0.000000,895.743164,667.948059,220.910858,...,0.000000,315.064514,232.140564,0.000000,0.000000,353.101288,90.148193,4.931998,0.000000,20.587511
2015,5.578628,0.000000,2.744384,1773.299072,0.000000,510.743042,0.000000,1136.955688,751.210693,305.372223,...,1.560413,338.189484,291.105652,0.056466,0.000000,384.711426,94.849472,5.262599,0.000000,25.042593
2016,7.125076,0.248937,4.782331,3306.877686,0.000000,541.683533,13.982696,1195.784180,894.969971,293.737579,...,6.649363,356.768433,356.734985,6.716151,10.471766,391.973236,106.654129,2263.000732,5.882514,24.095966
2017,7.585610,0.175167,5.272010,3572.213623,0.124469,554.622314,13.642684,1300.468750,965.250122,321.342438,...,6.828780,360.316132,324.120819,9.014268,12.149059,383.165283,95.756706,2391.040039,11.641469,23.038336


## June

In [37]:
pd.pivot_table(d[(d.month.isin([6]))], index='year', columns='family', values='sales', aggfunc='mean')

family,AUTOMOTIVE,BABY CARE,BEAUTY,BEVERAGES,BOOKS,BREAD/BAKERY,CELEBRATION,CLEANING,DAIRY,DELI,...,MAGAZINES,MEATS,PERSONAL CARE,PET SUPPLIES,PLAYERS AND ELECTRONICS,POULTRY,PREPARED FOODS,PRODUCE,SCHOOL AND OFFICE SUPPLIES,SEAFOOD
year,,,,,,,,,,,,,,,,,,,,,
2013,5.130247,0.000000,2.371605,1129.077759,0.000000,375.176361,0.000000,916.011108,364.751862,207.351959,...,0.000000,339.404175,188.101852,0.000000,0.000000,202.068756,87.659523,5.327778,0.000000,18.264797
2014,5.080247,0.000000,2.188272,1296.838257,0.000000,400.726837,0.000000,915.066040,673.620361,231.122894,...,0.000000,340.772186,240.088272,0.000000,0.000000,363.581573,89.812599,5.112963,0.000000,21.308594
2015,6.038889,0.127778,3.506173,2942.732178,0.000000,509.215179,13.651852,1090.089478,767.646912,303.845612,...,1.419753,348.774139,275.336426,5.438272,8.076543,380.570496,100.208641,2172.355957,1.230864,23.938612
2016,6.133333,0.269136,4.464198,2754.985107,0.000000,540.608459,13.706173,1087.906738,847.920349,275.458893,...,5.569136,344.164917,287.314209,5.841358,8.343827,366.322174,103.020401,2197.563477,1.519136,22.746851
2017,7.380247,0.182099,5.432099,3580.134521,0.037037,555.641479,13.797531,1228.699951,932.528381,317.703125,...,6.187654,388.383240,319.435181,9.216666,11.338889,406.581512,90.897163,2403.449707,1.577160,19.361589
